In [ ]:
import pandas as pd

data_frame = pd.read_csv("/home/cmlare/Data/RF Data/Processed/2018-06-01 to 2018-06-10_integrated_classified.csv")

link_file = pd.read_csv("/home/cmlare/Data/RF Data/Master Files/mapped_links.csv")

data_file["Baseline_ATTN"] = data_file["Baseline"].sub(data_file["RSL_MIN"])
data_file["B_ATTN_PathLen"] = data_file["Baseline_ATTN"].div(data_file["distance"])

set1 = set([])
nearby_links = list()

# Loop to calculate unique links
for index, row in data_file.iterrows():
   set1.add(row['ID']) 

#  find the nearby links
for element in set1:
    nearbylinks_list = list()
    row = pd.concat([link_file.loc[link_file.id_1 == element],link_file.loc[link_file.id_2 == element] ])
    long1 = row.longitude1.values[0]
    lat1 = row.latitude1.values[0]
    long2 = row.longitude2.values[0]
    lat2 = row.latitude2.values[0]
    
    for index2, row2 in link_file.iterrows():
        newlong1 = row2.longitude1
        newlat1 = row2.latitude1
        newlong2 = row2.longitude2
        newlat2 = row2.latitude2

#       Using the geopy library we are calculating the geodesic distance between two points.
#       rror of up to about 0.5%  
        distance1 = distance((lat1, long1), (newlat1, newlong1)).km
        distance2 = distance((lat1, long1), (newlat2, newlong2)).km
        distance3 = distance((lat2, long2), (newlat1, newlong1)).km
        distance4 = distance((lat2, long2), (newlat2, newlong2)).km
        
        condition = ((distance1 or distance2) and (distance3 or distance4))<= 5
        
        if (condition):
            nearbylinks_list.append(row2.id_1)
            nearbylinks_list.append(row2.id_2)
    nearby_links.append(nearbylinks_list)

In [ ]:
voted_classes = []
def voting (data_frame):
    iteration_check = 0
    size = data_frame["class"].size

    for index,row in data_frame.iterrows():
        print(iteration_check + " / " + size)
        iteration_check += 1
        current_id = row.ID
        timestamp = row.date_time
    
        index_from_ids = set1_tolist.index(current_id)
        get_nearbylinks = nearby_links[index_from_ids]

        if len(get_nearbylinks) <5:
            continue
        VOTED_CLASSES_LIST = []

        loop_breaker = 0 #limit the links to 10
    #     for loop to create the attenuation list for the selected link
        for link in get_nearbylinks:
            loop_breaker +=1
            row2 = data_frame.loc[data_frame.date_time == timestamp] # select all the  links for timpstamp
            row1 = row2.loc[row2.ID == link] # select the relevant link
            VOTED_CLASSES_LIST.append(row1["results_class"])
            if(loop_breaker > 10):
                break
        new_df["voted_class"] = VOTED_CLASSES_LIST
        winning_class = new_df.groupby('voted_class').size().idxmax()
        
        voted_classes.append(winning_class)
        
    data_frame["voted_class"] = voted_classes